In [27]:
import pandas as pd
import numpy as np
import os

In [28]:
df_ckplus_dataset = pd.read_csv("dataset/ckplus_dataset.csv")

In [29]:
def get_session(subject_id, session_id):
    return df_ckplus_dataset[(df_ckplus_dataset["subject_id"] == subject_id) & (df_ckplus_dataset["session_id"] == session_id)]

In [30]:
def read_emotion_features(emotion):
    distance_features = []
    angle_features = []
    with open(f"features/{emotion}.txt", "r") as file:
        for line in file:
            feature = line.strip().split()
            feature = [int(value) for value in feature[0][2:-1].split(",")]
            if len(feature) == 2:
                distance_features.append(feature)
            else:
                angle_features.append(feature)
    return distance_features, angle_features


def load_emotion_features():
    emotions = ["neutral", "anger", "disgust", "fear", "happiness", "sadness", "surprise"]
    emotion_features = {}
    for emotion in emotions:
        distance_features, angle_features = read_emotion_features(emotion)
        emotion_features[emotion] = (distance_features, angle_features)
    return emotion_features


emotion_features = load_emotion_features()

In [31]:
def euclidean_distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)


def smaller_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.arccos(cosine) * 180 / np.pi


def mid_point(a, b):
    return [(a[0] + b[0]) / 2, (a[1] + b[1]) / 2]


def get_face_width(data):
    point1 = data["landmark_2"].split(" ")
    point1 = [float(point1[0]), float(point1[1])]
    point2 = data["landmark_16"].split(" ")
    point2 = [float(point2[0]), float(point2[1])]
    return euclidean_distance(point1, point2)


def generate_distance_features(data, feature_emotion):
    distance_features_template = emotion_features[feature_emotion][0]

    distance_features = []
    for feature in distance_features_template:
        point1 = data[f"landmark_{feature[0]}"].split(" ")
        point1 = [float(point1[0]), float(point1[1])]
        point2 = data[f"landmark_{feature[1]}"].split(" ")
        point2 = [float(point2[0]), float(point2[1])]

        distance = euclidean_distance(point1, point2)
        distance_features.append(distance)

    if feature_emotion == "neutral":
        point1 = data["landmark_37"].split(" ")
        point1 = [float(point1[0]), float(point1[1])]
        point2 = data["landmark_46"].split(" ")
        point2 = [float(point2[0]), float(point2[1])]
        point3 = data["landmark_9"].split(" ")
        point3 = [float(point3[0]), float(point3[1])]

        distance = euclidean_distance(mid_point(point1, point2), point3)
        distance_features.append(distance)

    distance_features = np.array(distance_features)
    distance_features = distance_features / get_face_width(data)

    return distance_features


def generate_angle_features(data, feature_emotion):
    angle_features_template = emotion_features[feature_emotion][1]

    angle_features = []
    for feature in angle_features_template:
        point1 = data[f"landmark_{feature[0]}"].split(" ")
        point1 = [float(point1[0]), float(point1[1])]
        point2 = data[f"landmark_{feature[1]}"].split(" ")
        point2 = [float(point2[0]), float(point2[1])]
        point3 = data[f"landmark_{feature[2]}"].split(" ")
        point3 = [float(point3[0]), float(point3[1])]

        angle = smaller_angle(point1, point2, point3)
        angle_features.append(angle)

    angle_features = np.array(angle_features)
    angle_features = angle_features / 180

    return angle_features

In [32]:
def generate_features(data, feature_emotion, feature_type):
    features = np.array([])

    if feature_type == "distance" or feature_type == "both":
        distance_features = generate_distance_features(data, feature_emotion)
        features = np.concatenate((features, distance_features))

    if feature_type == "angle" or feature_type == "both":
        angle_features = generate_angle_features(data, feature_emotion)
        features = np.concatenate((features, angle_features))

    return features


def get_features_from_df(df, feature_emotion, feature_type):
    features = []
    for index, row in df.iterrows():
        features.append(generate_features(row, feature_emotion, feature_type))
    return np.array(features)

In [33]:
features = get_features_from_df(df_ckplus_dataset[:1], "neutral", "both")

In [35]:
features

array([[0.32036478, 0.30411235, 0.59663813, 0.47326867, 0.47553256,
        1.        , 0.20333555, 0.32334929, 0.75505264]])

In [36]:
row = df_ckplus_dataset.iloc[0]

In [38]:
row["landmark_37"], row["landmark_46"], row["landmark_9"]

('280.10863 206.43671', '459.57889 196.31247', '382.36266 428.5127')

In [41]:
row["landmark_2"], row["landmark_16"]

('216.89078 254.86311', '518.15853 252.06194')